In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sys
import time
from tqdm import  tqdm

## 1. Preprocessing Hackathon official data

In [ ]:
def create_sample_ecg_file():
    """Create a sample ECG file for testing"""
    print("Creating sample ECG data file...")
    
    # Generate synthetic ECG data
    time_points = 5000  # Shorter for demo
    t = np.linspace(0, 10, time_points)
    
    # Simulate ECG signal
    ecg_signal = (
        0.8 * np.sin(2 * np.pi * 1.2 * t) +  # Main heartbeat
        0.3 * np.sin(2 * np.pi * 5 * t) +    # P-wave component
        0.1 * np.sin(2 * np.pi * 15 * t) +   # QRS complex
        0.05 * np.random.normal(0, 1, time_points)  # Noise
    )
    
    # Save to CSV
    df = pd.DataFrame({'ecg_values': ecg_signal})
    df.to_csv('sample_ecg_data.csv', index=False)
    
    print(f"Sample ECG data saved to 'sample_ecg_data.csv' with {len(ecg_signal)} data points")
    
    # Plot the sample
    plt.figure(figsize=(30, 5))
    plt.plot(t[:1000], ecg_signal[:1000])  # Plot first 1000 points
    plt.title('Sample ECG Signal (First 1000 points)')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.savefig('sample_ecg_plot.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return 'sample_ecg_data.csv'

def convert_text_to_csv():
    """Convert text file with ECG values to CSV"""
    file_path = input("Enter path to your text file with ECG values: ").strip()
    
    try:
        # Try different delimiters
        for delimiter in [',', ' ', '\t', ';']:
            try:
                data = pd.read_csv(file_path, delimiter=delimiter, header=None)
                if data.shape[1] > 1 or len(data) > 10:  # Valid data found
                    break
            except:
                continue
        
        print(f"Loaded data with shape: {data.shape}")
        
        # If multiple columns, ask which one to use
        if data.shape[1] > 1:
            print(f"Found {data.shape[1]} columns. Which column contains ECG data?")
            for i in range(data.shape[1]):
                print(f"Column {i}: {data.iloc[:5, i].values}")
            
            col_idx = int(input("Enter column index (0-based): "))
            ecg_data = data.iloc[:, col_idx]
        else:
            ecg_data = data.iloc[:, 0]
        
        # Save as CSV
        output_file = file_path.replace('.txt', '_converted.csv').replace('.dat', '_converted.csv')
        ecg_df = pd.DataFrame({'ecg_values': ecg_data})
        ecg_df.to_csv(output_file, index=False)
        
        print(f"Converted data saved to: {output_file}")
        print(f"Data points: {len(ecg_data)}")
        
        return output_file
        
    except Exception as e:
        print(f"Error converting file: {e}")
        return None

def main():
    print("="*50)
    print("ECG DATA PREPARATION HELPER")
    print("="*50)
    
    print("\nChoose an option:")
    print("1. Create sample ECG data for testing")
    print("2. Convert your text/dat file to CSV")
    print("3. Exit")
    
    choice = input("\nEnter choice (1-3): ").strip()
    
    if choice == '1':
        create_sample_ecg_file()
    elif choice == '2':
        convert_text_to_csv()
    elif choice == '3':
        print("Goodbye!")
    else:
        print("Invalid choice")

if __name__ == "__main__":
    main()

### 2. Converting `dataset/drugdata/DICT_rank.xlsx` to `dataset/drugdata/DICT_rank.csv` for compatibility

In [ ]:
def convert_dictrank_to_csv():
    """
    Convert DICTrank Excel dataset to CSV with specific rules:
    1. Fill NA in Column G (Keywords) and Column H (DIC Severity Level)
    2. Skip rows where Trade Name or Active Ingredient(s) are blank
    """
    
    # File paths
    # Change the file name
    excel_file = "dataset/drugdata/DICT_rank.xlsx" 
    output_file = "dataset/drugdata/DICT_rank.csv"
    
    print("="*70)
    print("DICTrank Dataset - Excel to CSV Converter")
    print("="*70)
    
    # Check if Excel file exists
    if not Path(excel_file).exists():
        print(f"\n❌ Error: File '{excel_file}' not found!")
        print("\nPlease ensure the Excel file is in the same directory as this script.")
        print("Or update the 'excel_file' variable in the script with correct path.")
        return
    
    try:
        # Load Excel file
        print(f"\n📂 Loading Excel file: {excel_file}")
        df = pd.read_excel(excel_file, engine='openpyxl')
        print(f"✓ Loaded {len(df)} rows and {len(df.columns)} columns")
        
        # Display column names
        print(f"\n📋 Columns found: {list(df.columns)}")
        
        initial_rows = len(df)
        
        # Step 1: Remove rows where Trade Name OR Active Ingredient(s) are blank
        print("\n🔍 Checking for blank Trade Name or Active Ingredient(s)...")
        
        # Check for blank/null values in Trade Name and Active Ingredient(s)
        mask_trade_name = df['Trade Name'].isna() | (df['Trade Name'].astype(str).str.strip() == '')
        mask_active_ingredient = df['Active Ingredient(s)'].isna() | (df['Active Ingredient(s)'].astype(str).str.strip() == '')
        
        # Combine masks - remove if either is blank
        rows_to_remove = mask_trade_name | mask_active_ingredient
        blank_rows_count = rows_to_remove.sum()
        
        if blank_rows_count > 0:
            print(f"⚠️  Found {blank_rows_count} rows with blank Trade Name or Active Ingredient(s)")
            print(f"   These rows will be skipped.")
            df = df[~rows_to_remove].copy()
            print(f"✓ Removed {blank_rows_count} rows")
        else:
            print("✓ No blank Trade Name or Active Ingredient(s) found")
        
        # Step 2: Fill NA in Keywords (Column G)
        print("\n📝 Processing Keywords column...")
        if 'Keywords' in df.columns:
            missing_keywords = df['Keywords'].isna().sum()
            empty_keywords = (df['Keywords'].astype(str).str.strip() == '').sum()
            total_missing_keywords = missing_keywords + empty_keywords
            
            if total_missing_keywords > 0:
                print(f"   Found {total_missing_keywords} missing values in Keywords")
                df['Keywords'] = df['Keywords'].fillna('NA')
                df.loc[df['Keywords'].astype(str).str.strip() == '', 'Keywords'] = 'NA'
                print(f"✓ Filled with 'NA'")
            else:
                print("✓ No missing values in Keywords")
        else:
            print("⚠️  Keywords column not found")
        
        # Step 3: Fill NA in DIC Severity Level (Column H)
        print("\n📝 Processing DIC Severity Level column...")
        if 'DIC Severity Level' in df.columns:
            missing_severity = df['DIC Severity Level'].isna().sum()
            empty_severity = (df['DIC Severity Level'].astype(str).str.strip() == '').sum()
            total_missing_severity = missing_severity + empty_severity
            
            if total_missing_severity > 0:
                print(f"   Found {total_missing_severity} missing values in DIC Severity Level")
                df['DIC Severity Level'] = df['DIC Severity Level'].fillna('NA')
                df.loc[df['DIC Severity Level'].astype(str).str.strip() == '', 'DIC Severity Level'] = 'NA'
                print(f"✓ Filled with 'NA'")
            else:
                print("✓ No missing values in DIC Severity Level")
        else:
            print("⚠️  DIC Severity Level column not found")
        
        # Save to CSV
        print(f"\n💾 Saving to CSV: {output_file}")
        df.to_csv(output_file, index=False, encoding='utf-8')
        
        final_rows = len(df)
        
        # Summary
        print("\n" + "="*70)
        print("✅ CONVERSION COMPLETE!")
        print("="*70)
        print(f"📊 Summary:")
        print(f"   - Initial rows: {initial_rows}")
        print(f"   - Rows skipped (blank Trade Name/Active Ingredient): {blank_rows_count}")
        print(f"   - Final rows in CSV: {final_rows}")
        print(f"   - Output file: {output_file}")
        print("="*70)
        
        # Display sample
        print("\n📄 Sample of converted data (first 5 rows):")
        print(df.head().to_string())
        
    except FileNotFoundError:
        print(f"\n❌ Error: Could not find the file '{excel_file}'")
        print("Please check the file name and path.")
    except Exception as e:
        print(f"\n❌ Error occurred: {str(e)}")
        print("\nPlease check:")
        print("1. The Excel file format is correct (.xlsx)")
        print("2. The file is not open in another program")
        print("3. Required packages are installed: pandas, openpyxl")
        import traceback
        traceback.print_exc()

### 3. Creating chem pub name of drugs code

In [ ]:
# Load your existing data
# be sure the path is correct
df = pd.read_csv('dataset/drugdata/drug_smiles.csv') # Make sure this matches your file name

# Create lists to store results
drug_ids = []
drug_names = []

print(f"🔄 Fetching names for {len(df)} drugs... this may take a few minutes.")

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    did = row['drug_id']
    smiles = row['smiles']
    
    try:
        # Strategy 1: Try to search by SMILES (most accurate)
        compounds = pcp.get_compounds(smiles, namespace='smiles')
        
        if compounds:
            # Get the first synonym (usually the common name)
            # We limit synonyms to find a short one
            found_name = compounds[0].synonyms[0] if compounds[0].synonyms else compounds[0].iupac_name
        else:
            found_name = did # Fallback to ID if not found
            
    except Exception as e:
        found_name = did # Fallback on error
        
    drug_ids.append(did)
    drug_names.append(found_name)
    
    # Sleep briefly to be polite to the API
    time.sleep(0.2)

# Save the new mapping file
name_df = pd.DataFrame({'drug_id': drug_ids, 'drug_name': drug_names})
name_df.to_csv('drug_names.csv', index=False)

print("✅ Success! 'drug_names.csv' created.")
print(name_df.head())